In [1]:
import gc

import numpy as np
import pandas as pd
from random import sample

import hyperopt
from hyperopt import hp, Trials
from hyperopt.pyll.base import scope

from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from coremdlr.models import NetworkModel
from coremdlr.config import defaults

from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.datasets.utils import infer_test_wells

import matplotlib.pyplot as plt
%matplotlib inline

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3

In [2]:
WAVENET_SEARCH_SPACE = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : scope.int(hp.quniform('num_blocks', 2, 20, 2)),
        'block_filters' : scope.int(hp.quniform('block_filters', 16, 256, 16)),
        'residual_filters' : scope.int(hp.quniform('residual_filters', 8, 128, 8)),
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
    },
    'loss' : {'categorical_crossentropy': hp.uniform('cat_loss', 0.1, 1.0),
              'ordinal_squared_error': hp.uniform('ord_loss', 0.1, 1.0)}
}


fit_args = {
    'batch_size' : 16,
    'epochs' : 20,
    'class_weighted' : True
}
CHKPT_FNAME = 'temp_best_weights.h5'


def make_params_usable(params):
    """
    Convert params returned by hyperopt to usable `model_args`
    """
    p = {**WAVENET_SEARCH_SPACE, **params}
    p['network_args']['num_blocks'] = int(params['network_args']['num_blocks'])
    p['network_args']['block_filters'] = int(params['network_args']['block_filters'])
    p['network_args']['residual_filters'] = int(params['network_args']['residual_filters'])
    return p

In [ ]:
def run_experiment(num_train, max_evals=15, search_space=WAVENET_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=["pseudoGR"],
                              pseudoGR_args={'features' : ['mean', 'var'], 
                                             'per_channel' : True}) for s in train_sets[:2]]
    
    for dset in datasets:
        dset.load_or_generate_data()
    
    def train_model(model_config):
        
        acc_scores, f1_scores, log_losses = [], [], []
        
        for dset in datasets:
            
            model = NetworkModel(dset, model_args=model_config)
            
            chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
            earlystop_callback = EarlyStopping(monitor='val_loss', patience=3)
            fit_args['callbacks'] = [chkpt_callback, earlystop_callback]
            
            print(f'Training with config: {model_config}')
            
            model.fit(dset, **fit_args)
            
            model.network.load_weights(CHKPT_FNAME)
            
            y_proba = model.predict_proba(dset.X_test)
            y_pred = np.argmax(y_proba, -1)
            
            acc_scores.append(accuracy_score(dset.y_test, y_pred))
            
            f1_scores.append(f1_score(dset.y_test, y_pred, average='macro'))
            
            log_losses.append(log_loss(dset.y_test, y_proba, 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return {'loss' : np.mean(log_losses),
                'acc_scores' : acc_scores,
                'f1_scores' : f1_scores,
                'log_losses' : log_losses,
                'status' : hyperopt.STATUS_OK}
    
    
    trials = Trials()
    
    best_params = hyperopt.fmin(
        fn=train_model,
        space=WAVENET_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        trials=trials
    )
    
    return trials, best_params

In [ ]:
trails3, best_params3 = run_experiment(3)

In [ ]:
trails3.results

In [ ]:
best_params3

In [ ]:
trails3.best_trial

In [7]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    pred_dfs = []
    
    for i, train_set in enumerate(train_sets):
        
        gc.collect()
        
        dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['pseudoGR'], 
                             pseudoGR_args={'features' : ['mean', 'var'], 'per_channel' : True})
        
        dset.load_or_generate_data()
        
        model = NetworkModel(dset, model_args=model_config)
        
        chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_acc', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
        earlystop_callback = EarlyStopping(monitor='val_acc', patience=6)
        fit_args['callbacks'] = [chkpt_callback, earlystop_callback]
        
        model.fit(dset, verbose=False, **fit_args)
        
        model.network.load_weights(CHKPT_FNAME)
        
        #print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name)
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
        del dset; del model;
        
    return pd.concat(pred_dfs)

In [8]:
params = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 18,
        'block_filters' : 128,
        'residual_filters' : 32,
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam'
    },
    'loss' : {'categorical_crossentropy': 0.360198,
              'ordinal_squared_error': 0.42455}
}

preds_df_3 = pred_results_df(3, params)
preds_df_3.to_csv('pgr_wavenet_results_train_sets3_FINAL.csv')

[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('pseudoGR', (19294, 32, 8))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13034,)), ('top', (13034,)), ('base', (13034,)), ('pseudoGR', (13034, 32, 8))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-20-1Z  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1929,)), ('top', (1929,)), ('base', (1929,)), ('pseudoGR', (1929, 32, 8))]
Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('pseudoGR', (3105, 32, 8))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3881,)), ('top', (3881,)), ('base', (3881,)), ('pseudoGR', (3881, 32, 8))]
Loading Well:  204-20-6a  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), 

...

Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3881,)), ('top', (3881,)), ('base', (3881,)), ('pseudoGR', (3881, 32, 8))]
Loading Well:  204-20-6a  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), ('pseudoGR', (3542, 32, 8))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13034,)), ('top', (13034,)), ('base', (130

/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), ('pseudoGR', (3542, 32, 8))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('pseudoGR', (11210, 32, 8))]
Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1947,)), ('top', (1947,)), ('base', (1947,)), ('pseudoGR', (1947, 32, 8))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data


/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/ross/code/python/coremdlr/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-20-1Z  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1929,)), ('top', (1929,)), ('base', (1929,)), ('pseudoGR', (1929, 32, 8))]
Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('pseudoGR', (3105, 32, 8))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3881,)), ('top', (3881,)), ('base', (3881,

In [9]:
accuracy_score(preds_df_3.y_pred.values, preds_df_3.y_true.values)

0.5281981380843465

In [6]:
defaults.DEFAULT_TRAIN_PATH

PosixPath('/home/ross/Dropbox/core_data/facies/train_data')

In [7]:
import pandas as pd

old_df = pd.read_csv('/home/ross/Dropbox/core_data/model_results/pgr_wavenet_results_train_sets3_ALL.csv')

accuracy_score(old_df.y_pred.values, old_df.y_true.values)

0.5160835768076856